<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/KoGPT_%EC%9A%94%EA%B8%B0%EC%9A%94_%EB%8B%B5%EB%B3%80%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.7 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/생성 AI 모델링/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device




'cpu'

# 데이터 정제

In [5]:
yogiyo =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi_all.csv")


## 정규표현식

In [85]:
train= yogiyo[['고객리뷰','사장댓글']]

rename_cols = ['review', 'reply']
train.columns = rename_cols
train.head()

,review,reply
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...


In [86]:
review_list = []
for review in train['review']:
    if isinstance(review, str):

        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", review)

        # result = re.sub(r'\.{2,}', '.', result)
        # result = re.sub(r'\,{2,}', ',', result)
        # result = re.sub(r'\~{2,}', '~', result)
        # result = re.sub(r'\!{2,}', '!', result)
        # result = re.sub(r'\?{2,}', '?', result)
        # result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train['review'] = review_list
train.head()

<ipython-input-86-699989ab745b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['review'] = review_list


,review,reply
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...


In [87]:
reply_list = []
for reply in train['reply']:
    if isinstance(reply, str):

        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", reply)

        # result = re.sub(r'\.{2,}', '.', result)
        # result = re.sub(r'\,{2,}', ',', result)
        # result = re.sub(r'\~{2,}', '~', result)
        # result = re.sub(r'\!{2,}', '!', result)
        # result = re.sub(r'\?{2,}', '?', result)
        # result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

train['reply'] = reply_list
train.head()

<ipython-input-87-ac9c3a7eab2c>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'] = reply_list


,review,reply
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,"이렇게 저희 매장 찾아주셔서 감사드립니다!! 맛 뿐만 아니라 양도, 서비스도 모두..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 ...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!! 저희 매장의 메뉴를 제대로 즐기신...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었...


## 고객id 처리
- ~님 -> 고객님

In [88]:
train['reply'] = train['reply'].str.replace(r'\w+\s*님', '고객님', regex=True)


<ipython-input-88-ef52f5ba6171>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'] = train['reply'].str.replace(r'\w+\s*님', '고객님', regex=True)


In [89]:
customer_sentence_id = []

for review in reply_list:
        if '님' in review :
          customer_sentence_id.append(review)

In [90]:
customer_sentence_id

['이렇게 저희 매장 찾아주셔서 감사드립니다!!  맛 뿐만 아니라 양도, 서비스도 모두 최상의 품질을 드리려고 노력하고 있습니다  모든 직원들이 힘내서 고객님의 마음을 잡기 위해 계속 노력하겠습니다    항상 건강 챙기시고, 오늘도 즐거운 하루 보내시길 바라겠습니다                                                 ',
 '좋은 리뷰 덕분에 기분이 행복해지는 것 같아요   고객님께서 맛있다고 말씀 주시니 피로가 싹~ 사라지네요! 주방에서 힘든 일도 있고 늘 쉽지만은 않지만, 이런 리뷰 읽을 때 마다 보람차고 다시 시작할 수 있는 거 같습니다   정말 감사드려요    오늘도 화이팅 있게 마무리하시길 바라요 ',
 '수 많은 매장들을 헤치고 와주셔서 감사합니다   메뉴 그 자체로 행복을 드리고 싶었는데 성공이였을까요?? 저희 메뉴 칭찬해 주셔서 감사합니다 고객님!! 늘 어떻게하면 더 맛있게 드릴 수 있을까 고민 많이 하고 있어요~~항상 노력하고 발전하도록 하겠습니다    항상 같은 만족감 드릴 수 있도록 언제나 최선을 다해 운영 하겠습니다                                                 ',
 '저희 매장 찾아주신 것도 감사한데 리뷰까지..감동입니다     고객님들의 배고픔을 저희는 지켜만 볼 수 없습니다! 언제든지 행복한 배부름을 느끼게 해 드릴 수 있도록 저희가 있지 않겠습니까!! 매번 찾아 주셔도 매번 시켜 먹고 싶은 매장이 되겠습니다    하시는 일 모두 번창하시길 진심으로 바라고 있을게요                                                     ',
 '정성 가득한 리뷰 작성해주셔서 감사합니다   맛있는 시간이 되셨다니 정말 다행입니다   저희 매장은 재료부터 맛, 포장, 서비스 모든 면에 진심을 담아 고객님이 완벽하게 행복하실 수 있도록 노력하겠습니다   언제든 신호만 주시면 바로 달려가는 매장이 되겠습니다! 다음에 또 만나요!          

## 가게명 처리
- reply에 가게명이 들어가있는 경우에 store 칼럼에 가게명 기재, 모델이 함께 학습할 수 있도록 처리



In [91]:
ygy_store_lst = ["1988응답하라추억의옛날도시락-방이점", "워커스커피 로스터스", "떡깨비-가락점", "당치땡-헬리오시티점", "우시장국밥-석촌점",  "고돼지-송파점",  "마라의신마라탕&마라샹궈",  "춘리마라탕-송파점",  "손수경의육회담은연어-가락점",  "무국적식탁-송파점",  "청년치킨-삼전점",  "송파만다린",  "배달의쌀국수-송파점",  "닭강점-헬리오시티점",  "토핑폭탄김치찜&김치찌개-송파점",  "조형훈족발보쌈",  "여왕의브런치",  "모어댄버터",  "플러스82-송파점",  "더티베지",  "잭아저씨족발보쌈-본점",  "육회&연어-미아본점",  "갈비민족-본점",  "구구족-성신여대역점",  "피자보이시나-대학로점",  "왕빈자삼파전-미아본점",  "행복한찜닭-성북점",  "스시사소우",  "저팔계&중국&마라탕&덮밥&요리&전문점",  "명작파스타-성북점",  "올데이파스타-동대문구점",  "대한냉면-성북점",  "인생제육",  "고기듬뿍미트박스-성북점",  "피자스웨그-성북점",  "청춘식당묵은지김치찜",  "삽교원조두리곱창",  "뜸들이다-북서울꿈의숲점",  "더팔당매운오징어&닭발 김치찜",  "우리집반찬도시락-혜화막내딸지점",  "수피자",  "마라쿡",  "김포불닭발",  "헬로팬닭갈비&마약볶음밥",  "대찌 대파생고기김치찌개&정육왕국물두루치기",  "구래상회",  "승도리네곱도리탕",  "자성당",  "수락",  "나인곱창",  "어시장",  "돈까스회관",  "호랑이초밥",  "땡초곱창막창",  "피자이탈리",  "텐텐마라탕",  "과일에반하다",  "삼다수",  "병천토속순대",  "최고남제육명가",  "짱닭치킨",  "걸작떡볶이",  "직화삼겹직구삼",  "짜글이가돼었소",  "부대찌개자신있는집",  "토핑폭탄김치찜&김치찌개-부천점",  "1989마라탕-부천점",  "미친피자-부평점",  "천년닭강정-부천부평점",  "수상한삼겹살-부천점",  "수제죽전문점-본가진죽-본점",  "오늘은 분식",  "이끌림마라탕",  "탐나는피자-부평점",  "고기혁명-부천점",  "라화방마라탕-신중동점",  "전금례닭볶음탕-부천점",  "큰아들백순대",  "공주닭발-부천시청점",  "신자매김치찜&김치찌개",  "짱닭치킨-도촌점",  "이태리면가게-야탑점",  "폴트버거-판교점",  "랜돌프뉴욕페페로니-야탑점",  "홍싸롱-수제돈까스&파스타",  "쇼부덮밥",  "카산도-정자본점",  "부성초밥-미금본점",  "닭장수섭삼계탕&전기구이통닭",  "깨돌이김밥-미금점",  "울진죽변항",  "GTS버거-분당정자점",  "코브라독스-분당미금역점",  "마라하오-죽전점",  "땅끝수산",  "초밥대통령-용인수지점",  "올데이케밥&샐러드",  "야키토리 카마쿠라",  "비오키친-서현본점",  "부산오빠1인국밥-분당점",  "알촌",  "밈피자",  "고씨네카레",  "홍스족발",  "뜸들이다",  "감성커피",  "짚신스시",  "떡형",  "39도시락",  "피제이피자",  "황궁쟁반짜장",  "OK반점",  "모모타코야키&카페",  "꼬알라파이",  "진지덮밥",  "피자프라텔로",  "한식세끼1인김치찜&김치찌개",  "윤희횟집",  "시민보쌈족발&감자탕",  "카페봄봄",  "킹프레소빅와플",  "최가네한쌈",  "창타이누들",  "보돌미역",  "홍대개미"  ]


In [92]:
store_name = []

for store in ygy_store_lst:
    pat = re.split(r'[- &]', store)
    store_name.append(pat[0])

In [93]:
store_name

['1988응답하라추억의옛날도시락',
 '워커스커피',
 '떡깨비',
 '당치땡',
 '우시장국밥',
 '고돼지',
 '마라의신마라탕',
 '춘리마라탕',
 '손수경의육회담은연어',
 '무국적식탁',
 '청년치킨',
 '송파만다린',
 '배달의쌀국수',
 '닭강점',
 '토핑폭탄김치찜',
 '조형훈족발보쌈',
 '여왕의브런치',
 '모어댄버터',
 '플러스82',
 '더티베지',
 '잭아저씨족발보쌈',
 '육회',
 '갈비민족',
 '구구족',
 '피자보이시나',
 '왕빈자삼파전',
 '행복한찜닭',
 '스시사소우',
 '저팔계',
 '명작파스타',
 '올데이파스타',
 '대한냉면',
 '인생제육',
 '고기듬뿍미트박스',
 '피자스웨그',
 '청춘식당묵은지김치찜',
 '삽교원조두리곱창',
 '뜸들이다',
 '더팔당매운오징어',
 '우리집반찬도시락',
 '수피자',
 '마라쿡',
 '김포불닭발',
 '헬로팬닭갈비',
 '대찌',
 '구래상회',
 '승도리네곱도리탕',
 '자성당',
 '수락',
 '나인곱창',
 '어시장',
 '돈까스회관',
 '호랑이초밥',
 '땡초곱창막창',
 '피자이탈리',
 '텐텐마라탕',
 '과일에반하다',
 '삼다수',
 '병천토속순대',
 '최고남제육명가',
 '짱닭치킨',
 '걸작떡볶이',
 '직화삼겹직구삼',
 '짜글이가돼었소',
 '부대찌개자신있는집',
 '토핑폭탄김치찜',
 '1989마라탕',
 '미친피자',
 '천년닭강정',
 '수상한삼겹살',
 '수제죽전문점',
 '오늘은',
 '이끌림마라탕',
 '탐나는피자',
 '고기혁명',
 '라화방마라탕',
 '전금례닭볶음탕',
 '큰아들백순대',
 '공주닭발',
 '신자매김치찜',
 '짱닭치킨',
 '이태리면가게',
 '폴트버거',
 '랜돌프뉴욕페페로니',
 '홍싸롱',
 '쇼부덮밥',
 '카산도',
 '부성초밥',
 '닭장수섭삼계탕',
 '깨돌이김밥',
 '울진죽변항',
 'GTS버거',
 '코브라독스',
 '마라하오',
 '땅끝수산',
 '초밥대통령',
 '올데이케밥',
 '

In [94]:
store_etc_name = []

for store in ygy_store_lst:
    pat = re.split(r'[- &]', store)
    store_etc_name.append(pat[1:])

In [95]:
# etc_name #지점 지역 or 메뉴명

In [96]:
# 매장 이름별 컬럼 생성(안 그러면 반복문 돌릴 때 덮어쓰기가 됨)
for store in store_name:
    train[store] = train['reply'].apply(lambda x: store if store in x else '')

# 하나의 컬럼으로 합치고 필요 없는 컬럼 삭제
train['store'] = train[store_name].apply(lambda x: ''.join(x[x.notna()]), axis=1)
train = train.drop(columns=store_name) #
train.head()

<ipython-input-96-a4a0a922c0af>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[store] = train['reply'].apply(lambda x: store if store in x else '')
<ipython-input-96-a4a0a922c0af>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[store] = train['reply'].apply(lambda x: store if store in x else '')
<ipython-input-96-a4a0a922c0af>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

,review,reply,store
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,"이렇게 저희 매장 찾아주셔서 감사드립니다!! 맛 뿐만 아니라 양도, 서비스도 모두...",
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 ...,
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만...,
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!! 저희 매장의 메뉴를 제대로 즐기신...,
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었...,


In [98]:
mask = train['store'] != ''
train[mask]    #약 1/3 6330개 리뷰답변에 가게명 확인

,review,reply,store
16,저저번달에 맛있게 먹어서 또 시켰는데 고기 양이 엄청 줄었네요 ..된장찌개도 엄청...,저희 1988응답하라추억의옛날도시락의 메뉴를 주문해 주셔서 감사합니다. 양이 줄...,1988응답하라추억의옛날도시락
23,20분 이내 배달 알림 주시고는 한시간이 지나서야 받았습니다. 배달이 늦는 점에 대...,안녕하세요 1988응답하라추억의옛날도시락 입니다. 남겨 주신 리뷰 잘 읽어보았습니...,1988응답하라추억의옛날도시락
43,맛이나 양은 무난해요 그런데... 배달예정시간이 무색하게 늦게도착 차라리 배달 ...,1988응답하라추억의옛날도시락을 선택해 주셔서 감사합니다. 저희 매장에서는 배달...,1988응답하라추억의옛날도시락
47,"메인은 좋았는데, 아... 반찬이 다 쉬어 모두 버렸어요 한번 씹는데 촉감,맛이 ...",안녕하세요 1988응답하라추억의옛날도시락 입니다. 우선 감사하게도 저희 매장을 찾...,1988응답하라추억의옛날도시락
51,예정시간보다 30분이 지나서야 도착하고 음식도 다 식어서 왔네요,1988응답하라추억의옛날도시락을 선택해 주셔서 감사합니다. 저희 매장에서는 배달...,1988응답하라추억의옛날도시락
...,...,...,...
18502,"여기 미역국이 맛있어서 주문했네요~ 죽과 미역국, 어르신께서 맛있게 드셨습니다 감사...",안녕하세요 어르신이 맛있게 드셨다니 정말 기쁘네요 멋진 칭찬 남겨주셔서 정말...,보돌미역
18507,오늘 생일인데 더 잘먹었습니다,안녕하세요 다시한번 생일축하드립니다 소중한 생일식사에 보돌미역 수영점을 찾아...,보돌미역
18508,"부모님 보내드렸는데, 양도 넉넉하고 따뜻하게 배달되어 맛있게 드셨다네요. 감사합니다",안녕하세요 귀한 식사에 보돌미역 수영점을 찾아주셔서 정말 감사드립니다 고객...,보돌미역
18522,믿고먹는밥 찐맛탱 굿,찐맛탱집 홍대개미입니다 맛있게 드셔주셔서 항상 감사드립니다!!! 신메뉴3종...,홍대개미


## 맞춤법

In [104]:
!git clone https://github.com/ssut/py-hanspell.git

Cloning into 'py-hanspell'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 110 (delta 16), reused 9 (delta 9), pack-reused 88
Receiving objects: 100% (110/110), 22.30 KiB | 3.72 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [105]:
!cd py-hanspell

In [106]:
!pip install py-hanspell/

Processing ./py-hanspell
  Preparing metadata (setup.py) ... done
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4813 sha256=c81944f6ebae781de5be6bed1d70db4758a59ea36bd7746fa094f0d834f4a169
  Stored in directory: /root/.cache/pip/wheels/a3/a7/16/8db596361f402638188412108152b8fbf6db76c898e00fe02d
Successfully built py-hanspell


In [107]:
from hanspell import spell_checker

def correct_spell(text):
    if isinstance(text, str):  # 입력 데이터가 문자열인 경우에만 맞춤법 검사 수행
        corrected_text = spell_checker.check(text).checked
        return corrected_text
    else:
        return text

In [108]:
correct_spell(train['review'][0])

'사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱 맞춰서 보내주셨어요~~~~'

In [111]:
train.head()

,review,reply,store
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,"이렇게 저희 매장 찾아주셔서 감사드립니다!! 맛 뿐만 아니라 양도, 서비스도 모두...",
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 ...,
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만...,
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!! 저희 매장의 메뉴를 제대로 즐기신...,
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었...,


In [113]:
correct_spell('감사합니다!!! 다음에도 이용해주세용')

'감사합니다!!! 다음에도 이용해 주세요'

In [122]:
train['review'][:10].apply(correct_spell)

0    사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...
1                     리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!
2                   잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.
3                                    굿 말이 필요 없는 메뉴였습니다
4                                        맛있어요 양도 많고 좋음
5                                           양 많고 맛있었어요
6    처음 주문했는데 맛있어서 다 먹었어요   리뷰 묵사발도 꼭 드세요! 맛있게 잘 먹었...
7                           가성비 아주 만족합니다. 맛있게 잘 먹었습니다.
8                                          맛있게 먹었습니다!!
9    새벽부터 시켰는데 배송 시간 맞춰서 잘 와주셨고 도시락도 맛있고 묵사발도 맛있습니다...
Name: review, dtype: object

In [121]:
train['reply'][:10].apply(correct_spell)

0    이렇게 저희 매장 찾아주셔서 감사드립니다!!  맛뿐만 아니라 양도, 서비스도 모두 ...
1    좋은 리뷰 덕분에 기분이 행복해지는 것 같아요   고객님께서 맛있다고 말씀 주시니 ...
2    소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다.   저희를 찾아 주신 만...
3    수많은 매장들을 헤치고 와주셔서 감사합니다!!  저희 매장의 메뉴를 제대로 즐기신 ...
4    수많은 매장들을 헤치고 와주셔서 감사합니다   메뉴 그 자체로 행복을 드리고 싶었는...
5    저희 매장 찾아주신 것도 감사한데 리뷰까지.. 감동입니다     고객님들의 배고픔을...
6    정성 가득한 리뷰 작성해 주셔서 감사합니다   맛있는 시간이 되셨다니 정말 다행입니...
7    즐거운 순간을 저희와 함께해 주셔서 감사합니다   믿고 찾아 주셔서 진심으로 감사드...
8    귀한 발걸음 감사합니다!!  아낌없이 칭찬해 주셔서 기분이 좋습니다   저희 매장은...
9    귀한 발걸음 감사합니다!!  고객님의 리뷰는 저희를 성장시키는 원동력이 됩니다~~ ...
Name: reply, dtype: object

In [130]:
# train['review'] = train['review'].apply(correct_spell)
# train['reply'] = train['reply'].apply(correct_spell)

In [ ]:
train.to_csv(f"{DATA_PATH}train_preprocessed.csv")

# 데이터 분석/증강/변형

## 별점 비율 확인

In [ ]:
train = pd.read_csv(f"{DATA_PATH}train_preprocessed.csv")

In [125]:
train_rate = pd.concat([train, yogiyo['별점']],axis=1)
train_rate

,review,reply,store,별점
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,"이렇게 저희 매장 찾아주셔서 감사드립니다!! 맛뿐만 아니라 양도, 서비스도 모두 ...",,5
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 ...,,5
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만...,,5
3,굿 말이 필요 없는 메뉴였습니다,수많은 매장들을 헤치고 와주셔서 감사합니다!! 저희 매장의 메뉴를 제대로 즐기신 ...,,5
4,맛있어요 양도 많고 좋음,수많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는...,,5
...,...,...,...,...
18681,맛있게 잘 먹었어요,감사합니다!!! 다음에도 이용해주세용,,5
18682,요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다,죄송합니다 !! 다음에도 애용해주세용,,4
18683,배달 빨라요 맛있게 잘 먹었습니당 !,감사합니다!!! 다음에도 이용해주세용,,5
18684,잘 먹었어요~~~~~,감사합니다!!! 다음에도 이용해주세용,,5


In [131]:
for i in range(1, 6):
    num_rating = train_rate[train_rate['별점'] == i].shape[0]
    globals()[f'rate_{i}'] = train_rate[train_rate['별점'] == i]
    print(f"별점 {i}: {num_rating}, {round(num_rating / train_rate.shape[0] * 100)}%")

별점 1: 92, 0%
별점 2: 64, 0%
별점 3: 179, 1%
별점 4: 560, 3%
별점 5: 17791, 95%


In [132]:
rate_1

,review,reply,store,별점
96,리뷰 이벤트 한다지만 너무 아닌 거 같아서 적습니다. 배달이 20분 넘게 안오길래 ...,안녕하세요 1988응답하라추억의옛날도시락 입니다. 주신 리뷰 잘 읽었습니다. 뒤돌...,1988응답하라추억의옛날도시락,1
144,진짜리뷰안쓰는데 양이 2만원씩내고 먹을양은 절대아닙니다 장난하는것도아니고,솔직한 리뷰 남겨 주셔서 감사합니다. 저희 1988응답하라추억의옛날도시락은 언제...,1988응답하라추억의옛날도시락,1
252,만들어 놓은지 한참 된 커피 .배달 예정 시간 25분이 지났어요 .,안녕하세요 배달이 늦어져 죄송합니다. 한파로 길이 얼어서 고객님들이 상황이 여유치가...,,1
363,밥이 쉬었어요... 단무지 때문도 아니고 김이 많아서 그런것도 아니고 쉬어서 냄새가...,밥이 쉬다뇨 말도 안되는 소리좀 하지마세요. 진짜 먼 정신이 이상한 사람인가 하루에...,,1
401,잠실점에 처음 시켜먹었던게 맛있어서 주문했더니 돈 은 나갔는데 다시 앱을 들어와보니...,요기요가 주문어플에 렉이 자주 있어서.. 여러가지로 혼돈이 있어 정말 죄송하게 되었...,,1
...,...,...,...,...
16302,여기 원래 세트메뉴에 원래 탕수육 소스 안나오나요?? 맛있어서 가끔 시켜멌었는데 세...,죄송합니다 비네리는 주말이다보니 배달이많이밀려 실수가있었던모양입니다 다시한번사과의말...,,1
16401,하진짜 사진도 아깝네요 면이 젓가락으로 집으면 그냥으깨져서 먹을수가없네요 콜라는...,안녕하세요 우선 사과말씀부터드립니다 정말죄송합니다 오늘비가많이오다보니 고객님들이 ...,,1
17313,사장님 처럼 사장님만보기 기능있면좋겠지만만족하지만개선되었으면할때쓰고아니면안시키고말...,저희 윤희횟집을 찾아주셔서 감사드립니다 고객님께서 저희에게 충고해주신것에대해 감사드...,윤희횟집,1
17679,음료 맛도 별로였고 빨대도 안주고 마감전이라 그런지 샌드위치 시켰는데 먹다가 그냥...,고객님 먼저 죄송하단 말씀 드립니다 어제 저희 직원이 실수로 빨대를 넣지...,,1


## 긍부정 비율 확인
- 식당 별점 평가 사전학습 모델 : MLOpsEngineer/review-rate-prediction
- Label_0 = 1점, Label_1 = 2점, Label_2 = 3점, Label_3 = 4점, Label_4 = 5점


In [133]:
from transformers import pipeline

# 모델 불러오기
model_name = "MLOpsEngineer/review-rate-prediction"
classifier = pipeline("sentiment-analysis", model=model_name)


In [136]:
classifier = pipeline("sentiment-analysis", model=model_name)

In [137]:
classifier(review_text)[0]

NameError: ignored

# 데이터 학습

## 데이터셋 만들기

## 학습 loop함수화

## 모델 저장하기

# 검증 및 평가

##